### In this notebook, we evaluate the tuned helsinki model and comparing it translation with google translation with reference to fancy translation from scraped data.

First: Let's download the wanted libraires.
 

** Note: Don't forget to restart runtime after running this cell.**

We're using nltk 3.4 because it's the only version that works without errors on the metric we're using.

In [1]:
!pip install --quiet transformers==4.7.0
!pip install --quiet sentencepiece==0.1.95
!pip install --upgrade gupload
!pip install datasets 
!pip install nltk==3.4

     |████████████████████████████████| 2.5 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 48.7 MB/s 
     |████████████████████████████████| 895 kB 42.1 MB/s 
     |████████████████████████████████| 1.2 MB 5.3 MB/s 
     |████████████████████████████████| 81 kB 4.2 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.290 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 

In [1]:
import pandas as pd
import re
import string
import torch
from tqdm.notebook import tqdm
tqdm.pandas()
from sklearn.model_selection import train_test_split
import nltk

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


# getting tokenizer   
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ar")

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/801k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/917k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

We connect google drive to github to load the model. 
you can find the model here: [Model](https://drive.google.com/file/d/1y4atW_aZW6pfr6W57zsItZJdiwATFhrv/view?usp=sharing)

In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
model = torch.load("/content/gdrive/MyDrive/Offiicial Folder/Tasks/Task2_Helsinki/helsinki_fineTuned.pt")
model.eval()

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(62802, 512, padding_idx=62801)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(62802, 512, padding_idx=62801)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0): MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
   

In [5]:
test = pd.read_csv('/content/gdrive/MyDrive/Offiicial Folder/Tasks/Task2_Helsinki/google_Translated.csv')
test.head()

,Unnamed: 0,Unnamed: 0.1,AR,EN,google_translation
0,0,2856,لذلك الاحتمال اللاحق ، حسنًا؟,that posterior probability okay,هذا الاحتمال اللاحق بخير
1,1,4397,مثل الانحدار اللوجستي ،,like logistic regression would do,مثلما يفعل الانحدار اللوجستي
2,2,8279,، واه ، قم بتدريب الخوارزمية ،,and uh train the algorithm,واه تدريب الخوارزمية
3,3,8994,أه ، إذا تم إنشاء البيانات من Gaussian,Uh if the data is generated from a Gaussian but,أه إذا تم إنشاء البيانات من Gaussian لكن
4,4,9688,أم ، وما سنراه مع آلات المتجهات الداعمة هو أننا,Um and what we will see with support vector ma...,أم وما سنراه مع آلات المتجهات الداعمة هو أننا


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1681 entries, 0 to 1680
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          1681 non-null   int64 
 1   Unnamed: 0.1        1681 non-null   int64 
 2   AR                  1681 non-null   object
 3   EN                  1681 non-null   object
 4   google_translation  1681 non-null   object
dtypes: int64(2), object(3)
memory usage: 65.8+ KB


We need to make sure our data is clean and there is more than one word in every row to avoid errors with google-bleu metric.

In [7]:
test = test[test.AR.str.count(' ') > 1]
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1638 entries, 0 to 1680
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          1638 non-null   int64 
 1   Unnamed: 0.1        1638 non-null   int64 
 2   AR                  1638 non-null   object
 3   EN                  1638 non-null   object
 4   google_translation  1638 non-null   object
dtypes: int64(2), object(3)
memory usage: 76.8+ KB


now, let's evaluate our model on test set

In [8]:
from datasets import load_metric

metric = load_metric('google_bleu')

Downloading:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

In [11]:
source = []
predictions = []
references = []
for index,row in test.iterrows():
    encode = model.generate(**tokenizer.prepare_seq2seq_batch(row['EN'],return_tensors='pt').to('cuda'))
    output = tokenizer.batch_decode(encode,skip_special_tokens=True)[0]
    source.append(row['EN'])
    predictions.append(output.split())
    references.append([row['AR'].split()])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3261: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1839: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3

In [12]:
result = metric.compute(predictions=predictions, references=references)
result["google_bleu"]

0.040161282005585344

Now, let's see the results on google translation.

In [13]:
google = []
for index,row in test.iterrows():
    google.append(row['google_translation'].split())

In [14]:
result_google = metric.compute(predictions=google, references=references)
result_google["google_bleu"]

0.4413307840081192

Saving the final results in dataFrame.

In [21]:
# getting text
text_pred = []
for i in range(len(predictions)):
  text_pred.append(" ".join(predictions[i]))

In [23]:
final = pd.DataFrame(list(zip(source, test['AR'].tolist(),test['google_translation'].tolist(),text_pred)), columns =['EN', 'AR','Google','Helsinki'])

In [24]:
final.head(10)

,EN,AR,Google,Helsinki
0,that posterior probability okay,لذلك الاحتمال اللاحق ، حسنًا؟,هذا الاحتمال اللاحق بخير,هذا الاحتمال اللاحق ، حسنًا؟
1,like logistic regression would do,مثل الانحدار اللوجستي ،,مثلما يفعل الانحدار اللوجستي,كما سيفعل الانحدار اللوجستي
2,and uh train the algorithm,، واه ، قم بتدريب الخوارزمية ،,واه تدريب الخوارزمية,، واه ، درب الخوارزمية
3,Uh if the data is generated from a Gaussian but,أه ، إذا تم إنشاء البيانات من Gaussian,أه إذا تم إنشاء البيانات من Gaussian لكن,أه ، إذا تم إنشاء البيانات من غاوسي ، ولكن ،
4,Um and what we will see with support vector ma...,أم ، وما سنراه مع آلات المتجهات الداعمة هو أننا,أم وما سنراه مع آلات المتجهات الداعمة هو أننا,أم ، وما سنرى مع آلات المتجهات الداعمة هو أننا
5,finite horizon problem where the agent acts fo...,ومشكلة الأفق المحدودة حيث يعمل الوكيل إلى الأبد,مشكلة الأفق المحدود حيث يتصرف الوكيل إلى الأبد,مشكلة الأفق المحدود حيث يتصرف الوكيل إلى الأبد
6,I think that every time theres a major technol...,أعتقد أنه في كل مرة يحدث فيها خلل تقني كبير ،,أعتقد أنه في كل مرة يحدث خلل تقني كبير,أعتقد أنه في كل مرة يكون هناك اضطراب تكنولوجي ...
7,So your given the rules of the game,إذن ، لديك قواعد اللعبة المعطاة ،,حتى تعطى قواعد اللعبة,إذن ، نظرًا لقواعد اللعبة ،
8,If I walk that costs one minute okay,إذا مشيت سيكلفني دقيقة واحدة ، حسنًا؟,إذا مشيت سيكلف ذلك دقيقة واحدة على ما يرام,إذا سارت ، هذا يكلف دقيقة واحدة ، حسنًا؟
9,a matrix derivative notation that allows you t...,تدوينًا مشتقًا لمصفوفة يسمح لك باشتقاق,رمز مشتق من المصفوفة يسمح لك بالاشتقاق,تدوين مشتق مصفوفة يتيح لك اشتقاق


In [25]:
final.to_csv('test_results.csv')

### Conclusions:


*   Though for arabic speakers the model output and reference are almost the same, for the metric it perform poorly.
*   We need a more accurate metric for arabic that can capture semantic.

